In [1]:
import pandas as pd
import numpy as np
import os
import folium
from folium.plugins import MarkerCluster, MousePosition, MeasureControl, LocateControl
import base64
from folium import IFrame


In [2]:
df = pd.read_excel('roteiro_viagem.xlsx')
#Transformando as coordenadas em números
df.lat = df.lat.astype(float)
df.lon = df.lon.astype(float)

#Concatenação do local com sigla do estado
df['local2'] = df['local'] +', '+ df['estado'] 

#Criando uma lista de solicitação de rotas
directions = []
for i in list(df.local2):
    i = i.replace(' ','+')
    i = i.replace(',','')
    directions.append (f'https://www.google.com/maps/dir/?api=1&destination={i}')
df['directions'] = directions

In [3]:
path = 'C:\\Users\\fabio\\OneDrive\\Documentos\\Projetos\\Roteirizacao\\Viagem_parana\\tumb\\'
lista_arquivos = []
for i in range(len(os.listdir(path))):
    lista_arquivos.append(os.path.splitext(os.listdir(path)[i])[0])

lista_arquivos.sort(reverse=True)

m = folium.Map(location=[-26.0014823,-49.3047171], 
               zoom_start=7)


coords_list = df[['local','lat', 'lon','status','OBS', 'link', 'directions']].reset_index(drop = True)
point_layer = folium.FeatureGroup(name="Destinos")

fg = folium.FeatureGroup(name='Visão por cluster', show=False) # Habilita o cluster 
m.add_child(fg)
lista_de_locals = coords_list[['lat', 'lon']].values.tolist()
MarkerCluster(locations = lista_de_locals).add_to(fg) # Cria cluster com os locals


icon_shape = ['circle', 'circle-dot', 'doughnut', 'rectangle-dot', 'marker']

for local in list(coords_list.index):
        if coords_list['status'][local] == 'visitado':
            cor = 'green'
        elif coords_list['status'][local] == 'cancelado':
            cor = 'red'
        else:
            cor = 'blue'
                    
        if str(local) in lista_arquivos:  
            image = path+lista_arquivos[lista_arquivos.index(str(local))]+'.jpg' 
            #A primeira solução que encotrei... o trecho --> lista_arquivos.index(str(local)), retorna o índice do número encontrado, depois utilizo o próprio resultado com index
        else:
            image = path+'100.png'
            
              
       
        encoded = base64.b64encode(open(image, 'rb').read())
        font = 'Helvetica'
        titulo = f'''<p style="text-align: center;"><span style="font-family: Helvetica, serif; font-size: 21px;">{coords_list.local[local]}</span></p>'''
        descricao = f'''<p style="text-align: center;"><span style="font-family: {font}, serif; style="color:#34B75F" font-size: 13px;">{coords_list.OBS[local]} </span></p>'''
        informacoes = f'''<p style="text-align: center;"><a href={coords_list.link[local]} target="_blank" title="Informação detalhadas"><span style="font-family: {font}, serif; font-size: 13px;">{'Mais informações'}</span></a></p>'''
        rota = f'''<p style="text-align: center;"><a href={directions[local]} target="_blank" title="Rota de como chegar ao local"><span style="font-family: {font}, serif; font-size: 13px;">Como chegar? </span></a></p>'''
        
      
        html = ('''<h3>'''+titulo+
             '</h3> <p style="text-align: center;"><img src="data:image/JPG;base64,{}" ><br><p>'+
            descricao+
            informacoes+
            rota).format
        #O Iframe permite incluir um elemento HTML dentro de outro objeto HTML principal
        iframe = IFrame(html(encoded.decode('UTF-8')), width=350, height=550) # Dimensões do poupup
        popup = folium.Popup(iframe, max_width=600, sticky = True)        
        
        #Adicionar Marcadores            
        point_layer.add_child(
            folium.Marker(
                list(coords_list[['lat','lon']].loc[local]),
                fill= True,# Preenchimento
                weight =1,# Espessura da linha
                popup = popup,
                tooltip = str(coords_list['local'][local]),
                icon = folium.plugins.BeautifyIcon(border_color = 'white',
                                                   border_width = 0.2,
                                                   background_color = cor,
                                                   text_color ='white',
                                                   number = coords_list.index[local],
                                                   icon_shape=icon_shape[4]))
        ).add_to(m)    

# Adicionar Legenda
legend_html = """
<div style="position:fixed;
     bottom: 50px; 
     left: 50px; 
     width: 220px; 
     height: 90px;      
     background-color:#f5f5f5c2;
     z-index: 9999;
     font-size:15px;">
     &nbsp;<b>Legenda:</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:#34B75F"></i>&nbsp;local Visitado<br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:#3193CC"></i>&nbsp;local Não Visitado<br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:#FF5733"></i>&nbsp;Cancelado <br>     
</div>"""


m.get_root().html.add_child(folium.Element(legend_html))
folium.LayerControl().add_to(m)

# Mostrar sua localização no mapa
LocateControl(auto_start=False).add_to(m)

#Salva o mapa
m.save('mapa_viagem_com_fotos.html')
m